In [9]:
import gradio as gr
import hopsworks
import joblib
import pandas as pd

In [10]:
# Load .env variables in environment

%reload_ext dotenv
%dotenv

In [34]:
labels = ['Low', 'Medium', 'High']

project = hopsworks.login()
fs = project.get_feature_store()

dataset_api = project.get_dataset_api()
dataset_api.download("Resources/images/wine_df_recent.png", overwrite=True)
dataset_api.download("Resources/images/wine_confusion_matrix.png", overwrite=True)

monitor_fg = fs.get_or_create_feature_group(name="wine_predictions", version=1, primary_key=["datetime"],
                                            description="Wine quality Prediction/Outcome Monitoring")

history_df = monitor_fg.read()
last_prediction = history_df.tail(1)
last_prediction = last_prediction.to_dict(orient='records')[0]

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/209044
Connected. Call `.close()` to terminate connection gracefully.


Downloading: 0.000%|          | 0/21831 elapsed<00:00 remaining<?

Downloading: 0.000%|          | 0/14098 elapsed<00:00 remaining<?

Finished: Reading data from Hopsworks, using ArrowFlight (0.63s) 


In [35]:
last_prediction

{'prediction': 0, 'label': 1.0, 'datetime': '11/19/2023, 14:11:45'}

In [37]:
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            gr.Label("Today's Predicted")
            gr.Label(f"{labels[last_prediction['prediction']] + ' quality' if last_prediction is not None else 'No predictions yet'}")
        with gr.Column():
            gr.Label("Today's Actual quality")
            gr.Label(f"{labels[int(last_prediction['label'])] + ' quality' if last_prediction is not None else 'No predictions yet'}")
    with gr.Row():
        with gr.Column():
            gr.Label("Recent Prediction History")
            gr.Image("wine_df_recent.png", elem_id="recent-predictions")
        with gr.Column():
            gr.Label("Confusion Maxtrix with Historical Prediction Performance")
            gr.Image("wine_confusion_matrix.png", elem_id="confusion-matrix")

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
